In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.io import mmread
X = csr_matrix(mmread('test.x.mm'))
y = np.loadtxt('test.y.dat')

In [4]:
import sklearn.linear_model as lm
model = lm.LinearRegression(fit_intercept = False)
model.fit(X, y)
print "R2=%f" % model.score(X, y)

R2=1.000000
